<a href="https://colab.research.google.com/github/HuangMengyu/DeepInteraction/blob/main/assignment_4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install Transformers
# !pip install Datasets
# !pip install Evaluate

In [ ]:
from datasets import load_dataset

# file paths for train and eval dataset
train_file = "/content/drive/MyDrive/Colab Notebooks/pa4/train.csv"
eval_file = "/content/drive/MyDrive/Colab Notebooks/pa4/eval.csv"

# create train and eval set
train_data = load_dataset("csv", data_files=train_file)
eval_data = load_dataset("csv", data_files=eval_file)
print(train_data.shape, eval_data.shape)

In [ ]:
from transformers import AutoTokenizer

# Pretrained tokenizer from BERT
bert_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_helper(batch):
    return bert_tokenizer(batch['review'], padding=True, truncation=True)

train_data = train_data.map(tokenize_helper, batched=True)
eval_data = eval_data.map(tokenize_helper, batched=True)
print(train_data.shape, eval_data.shape)
print(train_data['train'][0])




In [ ]:
from transformers import AutoModelForSequenceClassification

# Classification model
BERT = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
BERT

In [ ]:
# Compute trainable parameters

def count_trainable_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("num of trainable parameter:", count_trainable_parameters(BERT))

In [ ]:
# evaluation helper

import evaluate

accuracy_scorer = evaluate.load('accuracy')

def evaluation_helper(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_scorer.compute(predictions=predictions, references=labels)

In [ ]:
# training parameters

from transformers import TrainingArguments

# get training args
def getTrainingArgs(output_dir, num_train_epochs, eval_strategy = 'epoch'):
    return TrainingArguments(output_dir=output_dir,
            num_train_epochs = epochs,
            eval_strategy = 'epoch',
            report_to= 'none'
            )


In [ ]:
# Trainer

from transformers import Trainer

def getTrainer(model, training_args, train_set, eval_set, compute_metrics):
    return Trainer(model=model,
          args=training_args,
          train_dataset=train_set,
          eval_dataset=eval_set,
          compute_metrics=compute_metrics,
      )


In [ ]:
# finetune the whole BERT

BERT = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

# hypers
output_dir = "/content/drive/MyDrive/Colab Notebooks/assignment_4" #The output directory where the model predictions and checkpoints will be written
epochs = 10 # num of epoches
train_args = getTrainingArgs(output_dir, epochs)
trainer = getTrainer(BERT, train_args, train_data['train'], eval_data, evaluation_helper)

# evaluating
result_before = trainer.evaluate() #accu before finetuning
print("accuracy before finetuning", result_before['eval_train_accuracy'])

# training
trainer.train()

# evaluating
result_after = trainer.evaluate() #accu after finetuning
print("accuracy after finetuning", result_after['eval_train_accuracy'])

